In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import random
import math
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from owlready2 import *
from datetime import date
from datetime import datetime

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation


In [2]:
onto = get_ontology("modelo_mcare.owl")
onto.load()
descriptors = pd.read_csv('scores.csv')
descriptors

,number,days,gender,age,afftype,melanch,inpatient,edu,marriage,work,madrs1,madrs2
0,condition_1,11,2,35-39,2.0,2.0,2.0,6-10,1.0,2.0,19.0,19.0
1,condition_2,18,2,40-44,1.0,2.0,2.0,6-10,2.0,2.0,24.0,11.0
2,condition_3,13,1,45-49,2.0,2.0,2.0,6-10,2.0,2.0,24.0,25.0
3,condition_4,13,2,25-29,2.0,2.0,2.0,11-15,1.0,1.0,20.0,16.0
4,condition_5,13,2,50-54,2.0,2.0,2.0,11-15,2.0,2.0,26.0,26.0
5,condition_6,7,1,35-39,2.0,2.0,2.0,6-10,1.0,2.0,18.0,15.0
6,condition_7,11,1,20-24,1.0,NaN,2.0,11-15,2.0,1.0,24.0,25.0
7,condition_8,5,2,25-29,2.0,NaN,2.0,11-15,1.0,2.0,20.0,16.0
8,condition_9,13,2,45-49,1.0,NaN,2.0,6-10,1.0,2.0,26.0,26.0
9,condition_10,9,2,45-49,2.0,2.0,2.0,6-10,1.0,2.0,28.0,21.0


In [11]:
educationMap = {
    "6-10": "highschoolComplete",
    "11-15": "collegeComplete",
    "16-20": "collegePostgraduated",
    "": "fundamentalComplete"
}

genderMap = {
    "1": "female",
    "2": "male"
}

marriageMap = {
    "1.0": "married",
    "2.0": "single"
}

workMap = {
    "1.0": "employed",
    "2.0": "unemployed"
}


mdd = onto.Disorder("Disorder-Major-Depression")
mdd.dsm5code = [ "296.xx" ]

for index, row in descriptors.iterrows():
    if(row["number"].startswith("condition_")):
        age = int(row["age"][:2])
        gender = genderMap.get(str(row["gender"]))
        education = educationMap.get(str(row["edu"]))
        maritalStatus = marriageMap.get(str(row["marriage"]))
        employmentStatus = workMap.get(str(row["work"]))
        
        person = onto.Person(row["number"])
        newProfile = onto.SociodemographicProfile(row["number"] + "_profile")
        newProfile.belongToPerson.append(person)
        newProfile.maritalStatus = maritalStatus
        newProfile.age = age
        newProfile.gender = gender
        newProfile.educationLevel = education
        newProfile.employmentStatus = employmentStatus
        
        
        newDiagnosis = onto.Diagnosis(row["number"] + "_diagnosis")
        newDiagnosis.diagnosisType = [ "new" ]
        newDiagnosis.diagnosisStatus = [ "confirmed" ]
        newDiagnosis.hasDisorder.append(mdd)
        
        assessment = onto.MedicalAssessment(row["number"] + "_assessment")
        context = onto.Context(row["number"] + "_context")
        
        assessment.hasSociodemographicProfile = newProfile
        assessment.hasDiagnosis.append(newDiagnosis)
        assessment.belongToContext.append( context )
        
        
        context.is_a.append(onto.Instant)
        context.happenedOn = date(2020, 2, 1)
        context.hasEvent = [ assessment ]
        context.happenedAtTime = [ context ]
        context.happenedWithPerson = [ person ]
        
        
onto.save(file = "savedmcare.xml", format = "rdfxml")
        
        

In [3]:
mdd_class = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#MajorDepression")
recurrent_mdd_class = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#RecurrentMajorDepression")
single_ep_mdd_class = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#SingleMajorDepression")
anxiety_class = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Anxiety")
general_anxiety_class = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#GeneralAnxiety")

diagnosis = {
    "new": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#NewDiagnosis"),
    "confirmed": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#ConfirmedDiagnosis"),
    "remission": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#RemissionDiagnosis")
}

mdd = onto.Disorder("Disorder-Recurrent-Major-Depression")
mdd.dsm5code = [ "296.3x" ]
mdd.is_a.append(mdd_class)
mdd.is_a.append(recurrent_mdd_class)

ppd = onto.Disorder("Disorder-PostPartum-Single-Episode-Depression")
ppd.dsm5code = [ "296.2x" ]
ppd.is_a.append(mdd_class)
ppd.is_a.append(single_ep_mdd_class)

anxiety = onto.Disorder("Disorder-General-Anxiety")
anxiety.dsm5code = [ "300.xx" ]
anxiety.is_a.append(anxiety_class)
anxiety.is_a.append(general_anxiety_class)

clonazepam = onto.Medication("Medication-Clonazepam")
sertralina = onto.Medication("Medication-Sertraline")
fluoxetina = onto.Medication("Medication-Fluoxetine")
escitalopram = onto.Medication("Medication-Escitalopram")
diazepam = onto.Medication("Medication-Diazepam")

pop_genders = []
pop_ages = []
pop_marital = []
pop_empl = []
pop_educ = []
pop_assessment_dates = []
pop_religion = []
pop_medication = []
pop_remission = []

random.seed( 42 )


age_group = { 
    "children": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Children"),
    "teenager": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Teenager"),
    "youngadult": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#YoungAdult"),
    "adult": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Adult"),
    "elderly": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Elderly"),
}

educ_level = {
    "collegeComplete": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#CollegeComplete"),
    "collegeIncomplete": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#CollegeIncomplete"),
    "collegePostgraduated": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#CollegePostgraduated"),
    "unknown": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#EduUnknown"),
    "fundamentalComplete": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#FundamentalComplete"),
    "fundamentalIncomplete": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#FundamentalIncomplete"),
    "highschoolComplete": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#HighschoolComplete"),
    "highschoolIncomplete": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#HighschoolIncomplete"),
    "neverStudied": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#NeverStudied"),
}

employ_status = {
    "employed": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Employed"),
    "housewife": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Housewife"),
    "other": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#OtherEmployment"),
    "retired": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Retired"),
    "selfEmployed": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Self-employed"),
    "sick-leave": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Sick_leave"),
    "student": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Student"),
    "unemployed": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Unemployed"),
}

gender = {
    "male": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Male"),
    "female": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Female"),
}

marital_status = {
    "cohabiting": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Cohabiting"),
    "divorced": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Divorced"),
    "married": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Married"),
    "single": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Single"),
    "widower": onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#Widower"),
}

def add_gender(genderType, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_genders.append(genderType)
    random.shuffle(pop_genders)

    
def add_age(age, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_ages.append(age)
    random.shuffle(pop_ages)
    
def add_marital_status(maritalType, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_marital.append(maritalType)
    random.shuffle(pop_marital)
    
def add_employment(employement, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_empl.append(employement)
    random.shuffle(pop_empl)

def add_education(education, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_educ.append(education)
    random.shuffle(pop_educ)
    
def add_religion(religion, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_religion.append(religion)
    random.shuffle(pop_religion)

def add_medication(medication, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_medication.append(medication)
    random.shuffle(pop_medication)
    
def add_assessment_date(date, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_assessment_dates.append(date)
    random.shuffle(pop_assessment_dates)
    
def add_remission(remission, n):
    number = int(math.ceil(n))
    for item in range(1, (number + 1), 1):
        pop_remission.append(remission)
    random.shuffle(pop_remission)
    
    
def add_inferred_classes_to_profile(profile):
    if profile.age < 13:
        profile.is_a.append(age_group["children"])
    elif profile.age < 18:
        profile.is_a.append(age_group["teenager"])
    elif profile.age < 25:
        profile.is_a.append(age_group["youngadult"])
    elif profile.age < 60:
        profile.is_a.append(age_group["adult"])
    elif profile.age >= 60:
        profile.is_a.append(age_group["elderly"])
        
    if profile.maritalStatus:
        profile.is_a.append(marital_status.get(profile.maritalStatus))
        
    if profile.gender:
        profile.is_a.append(gender.get(profile.gender))
        
    if profile.educationLevel:
        profile.is_a.append(educ_level.get(profile.educationLevel))
        
    if profile.employmentStatus:
        profile.is_a.append(employ_status.get(profile.employmentStatus))

def add_inferred_classes_to_diagnosis(toUpdate):
    if toUpdate.diagnosisType[0]:
        toUpdate.is_a.append(diagnosis.get(toUpdate.diagnosisType[0]))

    if toUpdate.diagnosisStatus[0]:
        toUpdate.is_a.append(diagnosis.get(toUpdate.diagnosisStatus[0]))
    
        
def cleanup_population():
    global pop_genders
    global pop_ages
    global pop_marital
    global pop_empl
    global pop_educ
    global pop_assessment_dates
    global pop_medication
    global pop_religion
    global pop_remission
    pop_genders = []
    pop_ages = []
    pop_marital = []
    pop_empl = []
    pop_educ = []
    pop_assessment_dates = []
    pop_medication = []
    pop_religion = []
    pop_remission = []

In [4]:
def generate_morrinhos_ce(number_of_indiv):
    
    Morrinhos = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-CE-Morrinhos")
    CE = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-CE")
    
    add_gender("female", (number_of_indiv * (47 / 72)))
    add_gender("male", (number_of_indiv * (25 / 72)))
    
    add_age(65, number_of_indiv)
    
    add_marital_status("married", (number_of_indiv * (23 / 72)))
    add_marital_status("single", (number_of_indiv * (2 / 72)))
    add_marital_status("divorced", (number_of_indiv * (2 / 72)))
    add_marital_status("widower", (number_of_indiv * (45 / 72)))
    
    add_education("neverStudied", (number_of_indiv * (27 / 72)))
    add_education("fundamentalIncomplete", (number_of_indiv * (28 / 72)))
    add_education("highschoolComplete", (number_of_indiv * (11 / 72)))
    add_education("collegeComplete", (number_of_indiv * (6 / 72)))
    
    add_employment("housewife", (number_of_indiv * (16 / 72)))
    add_employment("selfEmployed", (number_of_indiv * (22 / 72)))
    add_employment("retired", (number_of_indiv * (34 / 72)))
    
    add_assessment_date(datetime.fromisoformat("2020-02-02T09:00:20+01:00"), (number_of_indiv * (36 / 72)))
    add_assessment_date(datetime.fromisoformat("2019-11-04T12:05:23+01:00"), (number_of_indiv * (36 / 72)))
    root_for_id = "MorrinhosPerson"
    
    for item in range(1, (number_of_indiv + 1), 1):
        gender  = pop_genders.pop()
        age     = pop_ages.pop()
        marital = pop_marital.pop()
        education = pop_educ.pop()
        employment = pop_empl.pop()
        base_id = root_for_id + str(item)
        
        person = onto.Person(base_id)
        newProfile = onto.SociodemographicProfile(base_id + "_profile")
        newProfile.belongToPerson.append(person)
        newProfile.maritalStatus = marital
        newProfile.age = age
        newProfile.gender = gender
        newProfile.educationLevel = education
        newProfile.employmentStatus = employment
        
        add_inferred_classes_to_profile(newProfile)
        
        newDiagnosis = onto.Diagnosis(base_id + "_diagnosis")
        newDiagnosis.diagnosisType = [ "new" ]
        newDiagnosis.diagnosisStatus = [ "confirmed" ]
        newDiagnosis.hasDisorder.append(mdd)
        add_inferred_classes_to_diagnosis(newDiagnosis)
        
        assessment = onto.MedicalAssessment(base_id + "_assessment")
        context = onto.Context(base_id + "_context")
        
        assessment.hasSociodemographicProfile = newProfile
        assessment.hasDiagnosis.append(newDiagnosis)
        assessment.belongToContext.append( context )
        
        
        context.is_a.append(onto.Instant)
        context.happenedOn = pop_assessment_dates.pop()
        context.hasEvent = [ assessment ]
        context.happenedAtTime = [ context ]
        context.happenedWithPerson = [ person ]
        context.happenedAtLocation = [ Morrinhos, CE ]
        
        

In [5]:
def generate_itajauba_anxiety(number_of_indiv):
    
    Itajauba = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-MG-Itajauba")
    MG = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-MG")
    
    add_gender("female", number_of_indiv * (383 / 605))
    add_gender("male", number_of_indiv * (222 / 605))
    
    add_age(18, number_of_indiv * (57 / 569))
    add_age(22, number_of_indiv * (76 / 569))
    add_age(35, number_of_indiv * (99 / 569))
    add_age(45, number_of_indiv * (126 / 569))
    add_age(55, number_of_indiv * (112 / 569))
    add_age(65, number_of_indiv * (99 / 569))
    
    add_marital_status("married", number_of_indiv * (37 / 92))
    add_marital_status("single", number_of_indiv * (44 / 92))
    add_marital_status("divorced", number_of_indiv * (5 / 92))
    add_marital_status("widower", number_of_indiv * (5 / 92))
    add_marital_status("cohabiting", number_of_indiv * (1 / 92))
    
    add_education("unknown", number_of_indiv)
    
    add_employment("student", number_of_indiv * (26 / 86))
    add_employment("employed", number_of_indiv * (31 / 86))
    add_employment("housewife", number_of_indiv * (23 / 86))
    add_employment("retired", number_of_indiv * (6 / 86))
    
    add_medication(clonazepam, number_of_indiv * 0.2917)
    add_medication(sertralina, number_of_indiv * 0.2505)
    add_medication(fluoxetina, number_of_indiv * 0.1773)
    add_medication(escitalopram, number_of_indiv * 0.1506)
    add_medication(diazepam, number_of_indiv * 0.1299)
    
    add_remission("no_remission", (number_of_indiv * 0.5))
    add_remission("remission", (number_of_indiv * 0.5))
    
    add_assessment_date(datetime.fromisoformat("2020-02-02T09:00:20+01:00"), number_of_indiv * (36 / 72))
    add_assessment_date(datetime.fromisoformat("2019-11-04T12:05:23+01:00"), number_of_indiv * (36 / 72))
    root_for_id = "ItajubaPerson"
    
    for item in range(1, (number_of_indiv + 1), 1):

        gender  = pop_genders.pop()
        age     = pop_ages.pop()
        marital = pop_marital.pop()
        education = pop_educ.pop()
        employment = pop_empl.pop()
        assessment_date = pop_assessment_dates.pop()
        medication = pop_medication.pop()
        
                
        base_id = root_for_id + str(item)
        
        person = onto.Person(base_id)
        newProfile = onto.SociodemographicProfile(base_id + "_profile")
        newProfile.belongToPerson.append(person)
        newProfile.maritalStatus = marital
        newProfile.age = age
        newProfile.gender = gender
        newProfile.educationLevel = education
        newProfile.employmentStatus = employment
        
        add_inferred_classes_to_profile(newProfile)
        
        
        newDiagnosis = onto.Diagnosis(base_id + "_diagnosis")
        newDiagnosis.diagnosisType = [ "new" ]
        newDiagnosis.diagnosisStatus = [ "confirmed" ]
        newDiagnosis.hasDisorder.append(anxiety)
        newDiagnosis.hasPrescribedMedication.append(medication)
        add_inferred_classes_to_diagnosis(newDiagnosis)
        
        assessment = onto.MedicalAssessment(base_id + "_assessment")
        context = onto.Context(base_id + "_context")
        
        assessment.hasSociodemographicProfile = newProfile
        assessment.hasDiagnosis.append(newDiagnosis)
        assessment.belongToContext.append( context )
        
        
        context.is_a.append(onto.Instant)
        context.happenedOn = assessment_date
        context.hasEvent = [ assessment ]
        context.happenedAtTime = [ context ]
        context.happenedWithPerson = [ person ]
        context.happenedAtLocation = [ Itajauba, MG ]
        
        if pop_remission.pop() == "remission":
            remission_date = datetime.fromisoformat("2020-03-01T12:05:23+01:00")
            remissionDiagnosis = onto.Diagnosis(base_id + "_remission_diagnosis")
            remissionDiagnosis.diagnosisType = [ "new" ]
            remissionDiagnosis.diagnosisStatus = [ "remission" ]
            remissionDiagnosis.hasDisorder.append(anxiety)
            add_inferred_classes_to_diagnosis(remissionDiagnosis)
            
            remissionAssessment = onto.MedicalAssessment(base_id + "_remission_assessment")
            remissionContext = onto.Context(base_id + "_remission_context")

            remissionAssessment.hasSociodemographicProfile = newProfile
            remissionAssessment.hasDiagnosis.append(remissionDiagnosis)
            remissionAssessment.belongToContext.append( remissionContext )


            remissionContext.is_a.append(onto.Instant)
            remissionContext.happenedOn = remission_date
            remissionContext.hasEvent = [ remissionAssessment ]
            remissionContext.happenedAtTime = [ remissionContext ]
            remissionContext.happenedWithPerson = [ person ]
            remissionContext.happenedAtLocation = [ Itajauba, MG ]

        
def generate_itajauba_both(number_of_indiv):
    
    Itajauba = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-MG-Itajauba")
    MG = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-MG")
    
    add_gender("female", number_of_indiv * 0.6428)
    add_gender("male", number_of_indiv * 0.3572)
    
    add_age(18, number_of_indiv * 0.1033)
    add_age(22, number_of_indiv * 0.1296)
    add_age(35, number_of_indiv * 0.1730)
    add_age(45, number_of_indiv * 0.2135)
    add_age(55, number_of_indiv * 0.1927)
    add_age(65, number_of_indiv * 0.1879)
    
    add_marital_status("married", number_of_indiv * 0.3899)
    add_marital_status("single", number_of_indiv * 0.5006)
    add_marital_status("divorced", number_of_indiv * 0.057)
    add_marital_status("widower", number_of_indiv * 0.0399)
    add_marital_status("cohabiting", number_of_indiv * 0.0125)
    
    add_education("unknown", number_of_indiv)
    
    add_employment("student", number_of_indiv * 0.2499)
    add_employment("employed", number_of_indiv * 0.3704)
    add_employment("housewife", number_of_indiv * 0.3059)
    add_employment("retired", number_of_indiv * 0.0738)
    
    add_medication(clonazepam, number_of_indiv * 0.2917)
    add_medication(sertralina, number_of_indiv * 0.2505)
    add_medication(fluoxetina, number_of_indiv * 0.1773)
    add_medication(escitalopram, number_of_indiv * 0.1506)
    add_medication(diazepam, number_of_indiv * 0.1299)
    
    add_remission("no_remission", (number_of_indiv * 0.5))
    add_remission("remission", (number_of_indiv * 0.5))
    
    add_assessment_date(datetime.fromisoformat("2020-02-02T09:00:20+01:00"), number_of_indiv * 0.5)
    add_assessment_date(datetime.fromisoformat("2019-11-04T12:05:23+01:00"), number_of_indiv * 0.5)
    root_for_id = "ItajubaComorbidPerson"
    
    for item in range(1, (number_of_indiv + 1), 1):
        try:
            gender  = pop_genders.pop()
        except IndexError as error:
            print("Item " + str(item) + "gender error")

        try:
            age     = pop_ages.pop()
        except IndexError as error:
            print("Item " + str(item) + "age error")
            
        try:
            marital = pop_marital.pop()
        except IndexError as error:
            print("Item " + str(item) + "marital error")
        
        try:
            education = pop_educ.pop()
        except IndexError as error:
            print("Item " + str(item) + "education error")
        
        try:
            employment = pop_empl.pop()
        except IndexError as error:
            print("Item " + str(item) + "employment error")
        
        try:
            assessment_date = pop_assessment_dates.pop()
        except IndexError as error:
            print("Item " + str(item) + "date error")
        
        medication = pop_medication.pop()
        
        base_id = root_for_id + str(item)
        
        person = onto.Person(base_id)
        newProfile = onto.SociodemographicProfile(base_id + "_profile")
        newProfile.belongToPerson.append(person)
        newProfile.maritalStatus = marital
        newProfile.age = age
        newProfile.gender = gender
        newProfile.educationLevel = education
        newProfile.employmentStatus = employment
        
        add_inferred_classes_to_profile(newProfile)
        
        
        newDiagnosis = onto.Diagnosis(base_id + "_diagnosis")
        newDiagnosis.diagnosisType = [ "new" ]
        newDiagnosis.diagnosisStatus = [ "confirmed" ]
        newDiagnosis.hasDisorder.append(anxiety)
        newDiagnosis.hasDisorder.append(mdd)
        newDiagnosis.hasPrescribedMedication.append(medication)
        add_inferred_classes_to_diagnosis(newDiagnosis)
        
        assessment = onto.MedicalAssessment(base_id + "_assessment")
        context = onto.Context(base_id + "_context")
        
        assessment.hasSociodemographicProfile = newProfile
        assessment.hasDiagnosis.append(newDiagnosis)
        assessment.belongToContext.append( context )
        
        
        context.is_a.append(onto.Instant)
        context.happenedOn = assessment_date
        context.hasEvent = [ assessment ]
        context.happenedAtTime = [ context ]
        context.happenedWithPerson = [ person ]
        context.happenedAtLocation = [ Itajauba, MG ]
        
        if pop_remission.pop() == "remission":
            remission_date = datetime.fromisoformat("2020-03-01T12:05:23+01:00")
            remissionDiagnosis = onto.Diagnosis(base_id + "_remission_diagnosis")
            remissionDiagnosis.diagnosisType = [ "new" ]
            remissionDiagnosis.diagnosisStatus = [ "remission" ]
            remissionDiagnosis.hasDisorder.append(mdd)
            remissionDiagnosis.hasDisorder.append(anxiety)
            add_inferred_classes_to_diagnosis(remissionDiagnosis)
            
            remissionAssessment = onto.MedicalAssessment(base_id + "_remission_assessment")
            remissionContext = onto.Context(base_id + "_remission_context")

            remissionAssessment.hasSociodemographicProfile = newProfile
            remissionAssessment.hasDiagnosis.append(remissionDiagnosis)
            remissionAssessment.belongToContext.append( remissionContext )


            remissionContext.is_a.append(onto.Instant)
            remissionContext.happenedOn = remission_date
            remissionContext.hasEvent = [ remissionAssessment ]
            remissionContext.happenedAtTime = [ remissionContext ]
            remissionContext.happenedWithPerson = [ person ]
            remissionContext.happenedAtLocation = [ Itajauba, MG ]
    
    
def generate_itajauba_depression(number_of_indiv):
    
    Itajauba = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-MG-Itajauba")
    MG = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-MG")
    
    add_gender("female", number_of_indiv * 0.6592)
    add_gender("male", number_of_indiv * 0.3408)
    
    add_age(18, number_of_indiv * 0.1086)
    add_age(22, number_of_indiv * 0.1229)
    add_age(35, number_of_indiv * 0.1714)
    add_age(45, number_of_indiv * 0.2000)
    add_age(55, number_of_indiv * 0.1857)
    add_age(65, number_of_indiv * 0.2114)
    
    add_marital_status("married", number_of_indiv * 0.3692)
    add_marital_status("single", number_of_indiv * 0.5385)
    add_marital_status("divorced", number_of_indiv * 0.0615)
    add_marital_status("widower", number_of_indiv * 0.0154)
    add_marital_status("cohabiting", number_of_indiv * 0.0154)
    
    add_education("unknown", number_of_indiv)
    
    add_employment("student", number_of_indiv * 0.1613)
    add_employment("employed", number_of_indiv * 0.3871)
    add_employment("housewife", number_of_indiv * 0.3710)
    add_employment("retired", number_of_indiv * 0.0806)
    
    add_medication(clonazepam, number_of_indiv * 0.2917)
    add_medication(sertralina, number_of_indiv * 0.2505)
    add_medication(fluoxetina, number_of_indiv * 0.1773)
    add_medication(escitalopram, number_of_indiv * 0.1506)
    add_medication(diazepam, number_of_indiv * 0.1299)
    
    add_remission("no_remission", (number_of_indiv * 0.5))
    add_remission("remission", (number_of_indiv * 0.5))
    
    add_assessment_date(datetime.fromisoformat("2020-02-02T09:00:20+01:00"), number_of_indiv * 0.5)
    add_assessment_date(datetime.fromisoformat("2019-11-04T12:05:23+01:00"), number_of_indiv * 0.5)
    root_for_id = "ItajubaDepressivePerson"
    
    for item in range(1, (number_of_indiv + 1), 1):
        gender  = pop_genders.pop()
        age     = pop_ages.pop()
        marital = pop_marital.pop()
        education = pop_educ.pop()
        employment = pop_empl.pop()
        assessment_date = pop_assessment_dates.pop()
        medication = pop_medication.pop()
        base_id = root_for_id + str(item)
        
        person = onto.Person(base_id)
        newProfile = onto.SociodemographicProfile(base_id + "_profile")
        newProfile.belongToPerson.append(person)
        newProfile.maritalStatus = marital
        newProfile.age = age
        newProfile.gender = gender
        newProfile.educationLevel = education
        newProfile.employmentStatus = employment
        
        add_inferred_classes_to_profile(newProfile)
        
        
        newDiagnosis = onto.Diagnosis(base_id + "_diagnosis")
        newDiagnosis.diagnosisType = [ "new" ]
        newDiagnosis.diagnosisStatus = [ "confirmed" ]
        newDiagnosis.hasDisorder.append(mdd)
        newDiagnosis.hasPrescribedMedication.append(medication)
        add_inferred_classes_to_diagnosis(newDiagnosis)
        
        assessment = onto.MedicalAssessment(base_id + "_assessment")
        context = onto.Context(base_id + "_context")
        
        assessment.hasSociodemographicProfile = newProfile
        assessment.hasDiagnosis.append(newDiagnosis)
        assessment.belongToContext.append( context )
        
        
        context.is_a.append(onto.Instant)
        context.happenedOn = assessment_date
        context.hasEvent = [ assessment ]
        context.happenedAtTime = [ context ]
        context.happenedWithPerson = [ person ]
        context.happenedAtLocation = [ Itajauba, MG ]
        
        if pop_remission.pop() == "remission":
            remission_date = datetime.fromisoformat("2020-03-01T12:05:23+01:00")
            remissionDiagnosis = onto.Diagnosis(base_id + "_remission_diagnosis")
            remissionDiagnosis.diagnosisType = [ "new" ]
            remissionDiagnosis.diagnosisStatus = [ "remission" ]
            remissionDiagnosis.hasDisorder.append(mdd)
            add_inferred_classes_to_diagnosis(remissionDiagnosis)
            
            remissionAssessment = onto.MedicalAssessment(base_id + "_remission_assessment")
            remissionContext = onto.Context(base_id + "_remission_context")

            remissionAssessment.hasSociodemographicProfile = newProfile
            remissionAssessment.hasDiagnosis.append(remissionDiagnosis)
            remissionAssessment.belongToContext.append( remissionContext )


            remissionContext.is_a.append(onto.Instant)
            remissionContext.happenedOn = remission_date
            remissionContext.hasEvent = [ remissionAssessment ]
            remissionContext.happenedAtTime = [ remissionContext ]
            remissionContext.happenedWithPerson = [ person ]
            remissionContext.happenedAtLocation = [ Itajauba, MG ]

def generate_salvador_ppd(number_of_indiv):
    Salvador = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-BA-Salvador")
    BA = onto.search_one(iri = "http://www.unisinos.br/pipca/ontologies/eroutines#BR-BA")
    
    add_gender("female", number_of_indiv )
    
    add_age(17, number_of_indiv * 0.1667)
    add_age(23, number_of_indiv * 0.4667)
    add_age(35, number_of_indiv * 0.3666)
    
    add_marital_status("married", number_of_indiv * 0.1667)
    add_marital_status("single", number_of_indiv * 0.4333)
    add_marital_status("cohabiting", number_of_indiv * 0.4000)
    
    add_education("neverStudied", number_of_indiv * 0.0333)
    add_education("fundamentalIncomplete", number_of_indiv * 0.1333)
    add_education("fundamentalComplete", number_of_indiv * 0.0667)
    add_education("highschoolIncomplete", number_of_indiv * 0.2333)
    add_education("highschoolComplete", number_of_indiv * 0.5)
    add_education("collegeIncomplete", number_of_indiv * 0.0334)
    
    add_employment("student", number_of_indiv * 0.1333)
    add_employment("employed", number_of_indiv * 0.2333)
    add_employment("housewife", number_of_indiv * 0.50)
    add_employment("unemployed", number_of_indiv * 0.1333)
    
    add_medication(sertralina, number_of_indiv * 0.2505)
    add_medication(fluoxetina, number_of_indiv * 0.1773)
    add_medication(escitalopram, number_of_indiv * 0.1506)
    
    add_remission("no_remission", (number_of_indiv * 0.50))
    add_remission("remission", (number_of_indiv * 0.50))
    
    add_assessment_date(datetime.fromisoformat("2020-02-02T09:00:20+01:00"), number_of_indiv * 0.5)
    add_assessment_date(datetime.fromisoformat("2019-11-04T12:05:23+01:00"), number_of_indiv * 0.5)
    root_for_id = "SalvadorPPDPerson"
    
    for item in range(1, (number_of_indiv + 1), 1):
        try:
            gender  = pop_genders.pop()
        except IndexError as error:
            print("Item " + str(item) + "gender error")

        try:
            age     = pop_ages.pop()
        except IndexError as error:
            print("Item " + str(item) + "age error")
            
        try:
            marital = pop_marital.pop()
        except IndexError as error:
            print("Item " + str(item) + "marital error")
        
        try:
            education = pop_educ.pop()
        except IndexError as error:
            print("Item " + str(item) + "education error")
        
        try:
            employment = pop_empl.pop()
        except IndexError as error:
            print("Item " + str(item) + "employment error")
        
        try:
            assessment_date = pop_assessment_dates.pop()
        except IndexError as error:
            print("Item " + str(item) + "date error")
            
        try:
            medication = pop_medication.pop()
        except IndexError as error:
            medication = ""
                
        base_id = root_for_id + str(item)
        
        person = onto.Person(base_id)
        newProfile = onto.SociodemographicProfile(base_id + "_profile")
        newProfile.belongToPerson.append(person)
        newProfile.maritalStatus = marital
        newProfile.age = age
        newProfile.gender = gender
        newProfile.educationLevel = education
        newProfile.employmentStatus = employment
        
        add_inferred_classes_to_profile(newProfile)
        
        
        newDiagnosis = onto.Diagnosis(base_id + "_diagnosis")
        newDiagnosis.diagnosisType = [ "new" ]
        newDiagnosis.diagnosisStatus = [ "confirmed" ]
        newDiagnosis.hasDisorder.append(ppd)
        if medication != "":
            newDiagnosis.hasPrescribedMedication.append(medication)
            
        add_inferred_classes_to_diagnosis(newDiagnosis)
        
        assessment = onto.MedicalAssessment(base_id + "_assessment")
        context = onto.Context(base_id + "_context")
        
        assessment.hasSociodemographicProfile = newProfile
        assessment.hasDiagnosis.append(newDiagnosis)
        assessment.belongToContext.append( context )
        
        
        context.is_a.append(onto.Instant)
        context.happenedOn = assessment_date
        context.hasEvent = [ assessment ]
        context.happenedAtTime = [ context ]
        context.happenedWithPerson = [ person ]
        context.happenedAtLocation = [ Salvador, BA ]
        
        if pop_remission.pop() == "remission":
            remission_date = datetime.fromisoformat("2020-03-01T12:05:23+01:00")
            remissionDiagnosis = onto.Diagnosis(base_id + "_remission_diagnosis")
            remissionDiagnosis.diagnosisType = [ "new" ]
            remissionDiagnosis.diagnosisStatus = [ "remission" ]
            remissionDiagnosis.hasDisorder.append(ppd)
            add_inferred_classes_to_diagnosis(remissionDiagnosis)
            
            remissionAssessment = onto.MedicalAssessment(base_id + "_remission_assessment")
            remissionContext = onto.Context(base_id + "_remission_context")

            remissionAssessment.hasSociodemographicProfile = newProfile
            remissionAssessment.hasDiagnosis.append(remissionDiagnosis)
            remissionAssessment.belongToContext.append( remissionContext )


            remissionContext.is_a.append(onto.Instant)
            remissionContext.happenedOn = remission_date
            remissionContext.hasEvent = [ remissionAssessment ]
            remissionContext.happenedAtTime = [ remissionContext ]
            remissionContext.happenedWithPerson = [ person ]
            remissionContext.happenedAtLocation = [ Salvador, BA ]
        

In [6]:
#generate_morrinhos_ce(72)
#generate_itajauba_anxiety(605)

generate_morrinhos_ce(1200)
cleanup_population()
generate_itajauba_both(2000)
cleanup_population()
generate_itajauba_depression(3200)
cleanup_population()


#generate_morrinhos_ce(73)
#cleanup_population()
#generate_itajauba_both(116)
#cleanup_population()
#generate_itajauba_depression(315)
#cleanup_population()
#generate_itajauba_anxiety(532)
#cleanup_population()
#generate_salvador_ppd(30)

In [7]:
#onto.save(file = "simulated.xml", format = "rdfxml")
onto.save(file = "6400-people.xml", format = "rdfxml")

In [8]:
#sync_reasoner()

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp C:\Users\i824612\AppData\Local\Continuum\anaconda3\lib\site-packages\owlready2\hermit;C:\Users\i824612\AppData\Local\Continuum\anaconda3\lib\site-packages\owlready2\hermit\HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:///C:/Users/i824612/AppData/Local/Temp/tmp2ilznrb_


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 1239093: invalid continuation byte

In [9]:
mdd.is_a

[modelo_mcare.Disorder]

In [10]:
inferences.get_parents_of(mdd)
mdd.is_a

NameError: name 'inferences' is not defined

In [ ]:
samplePerson = onto.Person("ASamplePerson")

In [5]:
newProfile = onto.SociodemographicProfile("NewProfile")

In [8]:
newProfile.belongToPerson.append(samplePerson)

In [6]:
newProfile.maritalStatus = [ "single" ]

In [9]:
newProfile.age = 35

In [13]:
onto.save()

IndexError: list index out of range

In [14]:
onto.save(file = "savedmcare", format = "rdfxml")